## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current
0,0,Rotifunk,8.2333,-12.6833,86.07,64,65,9.19,SL,2021-04-10 18:04:41,broken clouds
1,1,Lyuban,52.7985,28.0048,46.00,68,16,8.59,BY,2021-04-10 18:04:41,few clouds
2,2,Praia Da Vitoria,38.7333,-27.0667,60.80,94,75,17.81,PT,2021-04-10 18:04:41,broken clouds
3,3,Tsurugi,36.4500,136.6333,37.40,93,0,3.44,JP,2021-04-10 18:04:42,clear sky
4,4,Fauske,67.2588,15.3918,35.01,90,100,10.25,NO,2021-04-10 18:04:42,snow


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current
0,0,Rotifunk,8.2333,-12.6833,86.07,64,65,9.19,SL,2021-04-10 18:04:41,broken clouds
9,9,Columbus,39.9612,-82.9988,78.01,41,90,18.41,US,2021-04-10 18:03:25,overcast clouds
16,16,Maxixe,-23.8597,35.3472,77.00,78,20,6.06,MZ,2021-04-10 18:04:44,few clouds
17,17,Mount Isa,-20.7333,139.5000,77.00,36,54,19.57,AU,2021-04-10 18:04:44,broken clouds
19,19,Vung Tau,10.3460,107.0843,81.14,80,73,22.75,VN,2021-04-10 18:04:14,broken clouds
20,20,Sao Filipe,14.8961,-24.4956,77.00,60,6,9.71,CV,2021-04-10 18:02:35,clear sky
24,24,San Antonio,29.4241,-98.4936,80.60,22,20,16.11,US,2021-04-10 18:02:34,few clouds
25,25,Dingle,10.9995,122.6711,75.54,82,57,5.59,PH,2021-04-10 18:01:52,broken clouds
43,43,Grand Gaube,-20.0064,57.6608,80.01,75,99,5.01,MU,2021-04-10 18:04:49,light rain
44,44,Paragominas,-2.9667,-47.4833,84.38,71,45,2.77,BR,2021-04-10 18:04:50,scattered clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       2
Date          0
Current       0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current
0,0,Rotifunk,8.2333,-12.6833,86.07,64,65,9.19,SL,2021-04-10 18:04:41,broken clouds
9,9,Columbus,39.9612,-82.9988,78.01,41,90,18.41,US,2021-04-10 18:03:25,overcast clouds
16,16,Maxixe,-23.8597,35.3472,77.00,78,20,6.06,MZ,2021-04-10 18:04:44,few clouds
17,17,Mount Isa,-20.7333,139.5000,77.00,36,54,19.57,AU,2021-04-10 18:04:44,broken clouds
19,19,Vung Tau,10.3460,107.0843,81.14,80,73,22.75,VN,2021-04-10 18:04:14,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
664,664,Bubaque,11.2833,-15.8333,77.63,71,7,13.18,GW,2021-04-10 18:07:29,clear sky
668,668,Sabang,5.8933,95.3214,83.86,70,70,8.23,ID,2021-04-10 18:07:30,broken clouds
670,670,Omboue,-1.5746,9.2618,78.40,83,100,4.76,GA,2021-04-10 18:07:31,overcast clouds
686,686,Mangrol,21.1167,70.1167,79.84,68,0,13.22,IN,2021-04-10 18:07:35,clear sky


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current,Lat,Lng,Hotel Name
0,Rotifunk,SL,86.07,broken clouds,8.2333,-12.6833,
9,Columbus,US,78.01,overcast clouds,39.9612,-82.9988,
16,Maxixe,MZ,77.00,few clouds,-23.8597,35.3472,
17,Mount Isa,AU,77.00,broken clouds,-20.7333,139.5000,
19,Vung Tau,VN,81.14,broken clouds,10.3460,107.0843,
20,Sao Filipe,CV,77.00,clear sky,14.8961,-24.4956,
24,San Antonio,US,80.60,few clouds,29.4241,-98.4936,
25,Dingle,PH,75.54,broken clouds,10.9995,122.6711,
43,Grand Gaube,MU,80.01,light rain,-20.0064,57.6608,
44,Paragominas,BR,84.38,scattered clouds,-2.9667,-47.4833,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [35]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current,Lat,Lng,Hotel Name
9,Columbus,US,78.01,overcast clouds,39.9612,-82.9988,DoubleTree Suites by Hilton Hotel Columbus Dow...
16,Maxixe,MZ,77.00,few clouds,-23.8597,35.3472,Oceano
17,Mount Isa,AU,77.00,broken clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
19,Vung Tau,VN,81.14,broken clouds,10.3460,107.0843,Green hotel
20,Sao Filipe,CV,77.00,clear sky,14.8961,-24.4956,Tortuga B&B
...,...,...,...,...,...,...,...
664,Bubaque,GW,77.63,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"
668,Sabang,ID,83.86,broken clouds,5.8933,95.3214,Hotel Citra
670,Omboue,GA,78.40,overcast clouds,-1.5746,9.2618,Hotel Olako
686,Mangrol,IN,79.84,clear sky,21.1167,70.1167,Hotel The Royal Honours


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))